## [RISS](http://www.riss.kr/index.do) 논문 스크래퍼
 - riss에서 제공하는 내보내기를 자동화하는 프로그램으로 아래의 기술을 사용하여 구현함
  1. [python 3.8](https://www.notion.so/python-3-8-5db4070824e742609563bad0c51e674f): 주말 교육에서 활용한 프로그램 언어 python 3.8 이용
  2. [chrome ](https://www.notion.so/chrome-07d2239fab50437fa1c707581962845d): Chrome 버전 87.0.4280.88 사용
  3. [webdriver](https://www.notion.so/webdriver-628d57ad689248138d9b0066afd1ae08)
    - https://chromedriver.chromium.org/downloads 에서 로컬PC에 설치된 Chrome 버전에 맞는 webdirver를 다운로드함
    - 크롬 드라이버 다운로드
    - 압축을 풀어 chromedriver.exe 파일을 C:\windows\system32 폴더에 복사
  4. [selenium](https://www.notion.so/selenium-976ca695cc1a463ba2a0ce727dfc3498)
    - 웹 크롤링을 위한 Python 프레임워크
    - (설치) pip install selenium
    - (설치확인) python -c "import selenium"

In [1]:
import selenium.webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import time
import atexit
import os
import pandas as pd
from glob import glob

In [2]:
#전역 변수 선언
TIMEOUT = 10
ROOT_DIR = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
DOWNLOAD_DIR = f'{ROOT_DIR}\\download\\riss'
print(DOWNLOAD_DIR)

#사이트 주소
URL = 'http://riss.kr'

#검색어
SEARCH_WORD = "자동차"

#로그인 정보 (본인 정보를 입력해야 함)
USER = ''
PASSWD = ''

c:\development\workspaces\python\weekend\AZPower\RPA\download


In [ ]:
#chrome webdriver를 생성한다.
# - download_path: 파일을 다운로드 받을 위치
# - headless: True이면 브라우저 화면 없이 background로 처리
def create_chrome(download_path, headless=False) :
    options = ChromeOptions()
    if headless:
        options.add_argument('headless')
        options.add_argument('--disable-gpu')
    options.add_experimental_option('prefs', {
        'download.default_directory' : download_path,
        'download.prompt_for_download' : False
    })

    chrome = Chrome(chrome_options = options)

    if headless :
        _enable_download_on_chrome_headless(chrome, download_path)

    #프로세스 종료시 크롬을 종료시키는 로직
    atexit.register(_close_chrome(chrome))
    
    return chrome

#headless=True 일때 다운로드 안되는(크롬 보안문제) 문제 해결
def _enable_download_on_chrome_headless(browser, download_dir) :
    browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')

    params = {
        'cmd': 'Page.setDownloadBehavior',
        'params': {
            'behavior': 'allow',
            'downloadPath': download_dir
        }
    }
    browser.execute("send_command", params)

#크롬 종료~
def _close_chrome(chrome) :
    def close() :
        chrome.close()
    return close

def chrome_wait(wait, condition):
    try:
        element = wait.until(condition)
        print("[DEBUG] 로딩 완료")
        return element
    except TimeoutException:
        print(f"[DEBUG] 페이지 로딩 시간 초과")

In [ ]:
# 사용자 로그인, 키워드 검색 후 내보내기 할 목록에 대한 옵션을 설정한다.
def prepare_download() :
    print("[INFO] 사이트 이동")
    browser.get(URL)

    #페이지 로드될 때까지 기다림 
    chrome_wait(wait, EC.element_to_be_clickable((By.XPATH, '//li[@id="LoginTopli0"]/a')))

    #로그인 버튼 클릭
    print("[INFO] 로그인 버튼 클릭")
    browser.find_element_by_xpath('//li[@id="LoginTopli0"]/a').click()

    #ID 입력
    idTag = browser.find_element_by_id('id01')
    idTag.clear()
    idTag.send_keys(USER)

    #패스워드 입력 후 엔터
    pwTag = browser.find_element_by_id('id02')
    pwTag.clear()
    pwTag.send_keys(PASSWD)
    pwTag.send_keys(Keys.RETURN)

    print("[INFO] 로그인 정보 입력 후 로그인 시작")

    #페이지 로드될 때까지 기다림 
    chrome_wait(wait, EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btnSearch')))


    #검색조건 입력 후 검색버튼 클릭
    print("[INFO] 검색시작")
    search = browser.find_element_by_id('query')
    search.clear()
    search.send_keys(SEARCH_WORD)
    search.send_keys(Keys.RETURN)

    #페이지 로드될 때까지 기다림 
    chrome_wait(wait, EC.element_to_be_clickable((By.XPATH, '//li[@class="tabM1 " or @class="tabM1 on"]/a')))

    print("[INFO] 국내학술논문 클릭")
    #국내학술논문 클릭
    browser.find_element_by_xpath('//li[@class="tabM1 " or @class="tabM1 on"]/a').click()

    print("[INFO] 100개씩 출력으로 조회")
    chrome_wait(wait, EC.presence_of_element_located((By.ID, 'sortSelect2_top')))
    #100개씩 출력 선택 및 조회버튼 클릭
    browser.find_element_by_xpath('//label[@for="sortSelect2_top"]').click()
    browser.find_element_by_xpath('//div[@class="optList on"]/ul/li/a[text()="100개씩 출력"]').click()
    browser.find_element_by_css_selector('button.listSearch').click()

    print("[INFO] 작성언어 한국어 선택 및 실행")
    chrome_wait(wait, EC.presence_of_element_located((By.ID, 're_a_kor70')))
    browser.find_element_by_xpath('//label[@for="re_a_kor70"]').click()
    time.sleep(0.5)
    browser.find_element_by_class_name("btnAction").click()

In [ ]:
# 내보내기 버튼을 클릭하여 팝업을 조회하고 
# 옵션 선택 후 100개씩 다운로드 한다.
def download_abstract(pageIndex, wait) : 

    print("[INFO] --전체목록 선택")
    chrome_wait(wait, EC.element_to_be_clickable((By.XPATH, '//div[@class="resultTop1"]/ul/li[1]/a')))

    #전체항목 선택
    browser.find_element_by_xpath('//label[@for="allchk3"]/span').click()

    print("[INFO] --내보내기 버튼 클릭")
    #내보내기 버튼 클릭
    browser.find_element_by_xpath('//div[@class="resultTop1"]/ul/li[1]/a').click()

    #내보내기 팝업으로 포커스를 이동
    for handler in browser.window_handles:
        if handler != browser.current_window_handle :
            browser.switch_to_window(handler)
            break

    #페이지 로드될 때까지 기다림 
    print("[INFO] --다운로드 조건 선택")
    chrome_wait(wait, EC.element_to_be_clickable((By.CLASS_NAME, "btnType1")))

    #다운로드 조건 선택
    browser.find_element_by_xpath('//label[@for="radio-3"]').click()
    browser.find_element_by_xpath('//label[@for="radio-8"]').click()
    browser.find_element_by_xpath('//label[@for="radio-12"]').click()

    #다운로드 버튼 클릭
    print("[INFO] --다운로드 시작")
    browser.execute_script("javascript:f_submit();")

    #다운로드가 완료되었는지 기다림
    fileName = os.path.join(DOWNLOAD_DIR, "myCabinetExcelData.xls")
    accumulateTime = 1;
    while not os.path.exists(fileName) :
        time.sleep(1)

    #파일이 존재하는지 확인하고 파일명 변경
    newFileName = os.path.join(DOWNLOAD_DIR, f"excel\\abstract_{pageIndex + 1}.xls")
    if(os.path.isfile(fileName) and os.path.getsize(fileName) > 0) :
        os.rename(fileName, newFileName)
        print("[INFO] --다운로드 파일 확인 완료")

        #xls to csv 파일 변경
        header = False
        if pageIndex == 0 : 
            header = True
        csvFilename = os.path.join(DOWNLOAD_DIR, f"csv\\abstract_{pageIndex + 1}.csv")
        read_file = pd.read_excel(newFileName)
        read_file.to_csv(csvFilename, index = None, header = header)
        print("[INFO] --CSV 파일 변환 완료")

    else :
        print(f"[INFO] --다운로드 파일 확인 실패 [{newFileName}]")
    
    #팝업 닫기
    browser.close()
    #원래 페이지로 포커스 이동
    browser.switch_to_window(browser.window_handles[0]) 
    
    time.sleep(1)
    #전체 선택 해제
    browser.find_element_by_xpath('//label[@for="allchk3"]/span').click()

In [ ]:
## 페이지별로 다운로드 반복 구간

cntPerPage = 100
totalCnt = 50614 #한국어 논문 개수
pageCnt = totalCnt // cntPerPage + 1

print(f"총 자료: {totalCnt}")
print(f"총 페이지: {pageCnt}")

#브라우저 생성 및 사이트 이동
browser = create_chrome(download_path=DOWNLOAD_DIR)
wait = WebDriverWait(browser, TIMEOUT)

prepare_download()
#page는 0부터 시작함
for pageIndex in range(pageCnt):
    startCnt = pageIndex * 100
    print(f'[INFO] {pageIndex + 1} page 수집합니다( goPage({startCnt}) )')
    if startCnt != 0 :
        browser.execute_script(f'goPage({startCnt})')
    download_abstract(pageIndex, wait)

#브라우저 종료
browser.close()